In [1]:
!pip install google-api-python-client
!pip install pandas
!pip install IPython


In [2]:
from googleapiclient.discovery import build 
import os
import pandas as pd
from IPython.display import JSON

# 

In [3]:
# reading api_key from file
api_key = ""
with open("key.txt","r") as f:
    api_key = f.read()

In [4]:
    # Get credentials and create an API client

api_service_name = "youtube"
api_version = "v3"
youtube = build (
    api_service_name, api_version, developerKey=api_key )
channel_id = "UCX6OQ3DkcsbYNE6H8uQQuVA"

In [5]:
# -*- coding: utf-8 -*-

def get_channels(youtube,channel_id):
    total_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    

    # loop through items
    for item in response['items']:
        data = {
                "Channel_Name" : item['snippet']['title'],
                "Suscribers" : item['statistics']['subscriberCount'],
                "View" : item['statistics']['viewCount'],
                "TotalVideo" : item['statistics']['videoCount'],
                "PlaylistId" : item['contentDetails']['relatedPlaylists']['uploads']
               }

    total_data.append(data)
    return pd.DataFrame(total_data)

def get_videos(youtube, video_id):
    video_ids = []
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=video_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=video_id,
            maxResults = 50,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        next_page_token = response.get('nextPageToken')
        # print(len(video_ids))
        
    return video_ids

channel_data = get_channels(youtube, channel_id)
video_id = str(channel_data['PlaylistId'][0]).strip()
video_ids =  get_videos(youtube, video_id)


# 

In [6]:
def get_video_data(video_ids):
    # Get video data for the given video IDs in chunks of 50
    all_video_data = []
    
    for i in range(0, len(video_ids), 50):
        batch_ids = video_ids[i:i + 50]
        
        # Request video details in chunks
        video_response = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=",".join(batch_ids)
        ).execute()
        
        # Extend the result into all_video_data
        all_video_data.extend(video_response.get("items", []))
    
    return all_video_data


In [7]:
response = get_video_data(video_ids)
JSON(response)



<IPython.core.display.JSON object>

In [8]:
#create a dictionary which will store columns of our data
my_video_data = {
    'video_id' : [],
    'channel_title' : [],
    'title' : [],
    'description' : [],
    'published_at' : [],
    'view_count' : [],
    'like_Count' : [],
    'favourite_Count' : [],
    'duration' : [],
    'caption' : [],
    'defination' : [],
    'commentCount' : []
}


### ->inserting data in dataset

In [9]:
for item in response:
    my_video_data['video_id'].append(item.get('id', '0'))
    my_video_data['channel_title'].append(item['snippet'].get('channelTitle', '0'))
    my_video_data['title'].append(item['snippet'].get('title', '0'))
    my_video_data['description'].append(item['snippet'].get('description', '0'))
    my_video_data['like_Count'].append(item['statistics'].get('likeCount', '0'))
    my_video_data['favourite_Count'].append(item['statistics'].get('favoriteCount', '0'))
    my_video_data['duration'].append(item['contentDetails'].get('duration', '0'))
    my_video_data['caption'].append(item['contentDetails'].get('caption', '0'))
    my_video_data['defination'].append(item['contentDetails'].get('definition', '0'))
    my_video_data['view_count'].append(item['statistics'].get('viewCount', '0'))
    my_video_data['published_at'].append(item['snippet'].get('publishedAt', '0'))
    my_video_data['commentCount'].append(item['statistics'].get('commentCount', '0'))


In [10]:
for key, value in my_video_data.items():
    print(f"{key}: {len(value)}")

video_id: 863
channel_title: 863
title: 863
description: 863
published_at: 863
view_count: 863
like_Count: 863
favourite_Count: 863
duration: 863
caption: 863
defination: 863
commentCount: 863


### ->Displaying data

In [11]:
df = pd.DataFrame(my_video_data)
df

,video_id,channel_title,title,description,published_at,view_count,like_Count,favourite_Count,duration,caption,defination,commentCount
0,7K-yxGWYsnk,MrBeast,"$10,000 Human Shuffleboard",,2025-04-22T16:30:14Z,96792587,1709805,0,PT40S,false,hd,2817
1,Tbn4cYCaZCk,MrBeast,"Drift This Car, Win $10,000",,2025-04-16T17:00:15Z,230744065,3735813,0,PT20S,false,hd,6937
2,figrl0aOULs,MrBeast,Watch This Video To Feed 1 Person In Need,Buying 1 Hoodie = 100 Meals for people in need...,2025-04-12T16:00:01Z,45616652,3374387,0,PT10M29S,true,hd,64354
3,Uxc84UUrVHk,MrBeast,You Can Stop Child Labor,,2025-04-11T16:00:01Z,53745586,2411453,0,PT58S,false,hd,14639
4,BzKaMelDzqo,MrBeast,How Fast Can I Go?,New Merch - https://mrbeast.store\n\nCheck out...,2025-04-05T16:00:12Z,111796791,2085955,0,PT18S,false,hd,7072
...,...,...,...,...,...,...,...,...,...,...,...,...
858,7qj3nuF9Dzw,MrBeast,Most Epic minecraft skin EVER (Psy),Psy in minecraft!!! drop a like for psy's mo...,2013-01-13T01:59:21Z,1174248,42867,0,PT31S,false,hd,3586
859,Y74b7WlcEpk,MrBeast,More birds IN MINECRAFT!!,Basically what this mod does is adds more bird...,2013-01-12T23:35:45Z,1313200,48084,0,PT2M6S,false,hd,3809
860,Z8nEEdXTaX0,MrBeast,Boxy item mod Minecraft. EPIC,At the begining i said i was mrbeast6000.... i...,2013-01-12T22:34:11Z,1546959,56682,0,PT1M30S,false,hd,4272
861,jP82d277Cc8,MrBeast,Harry Potter Mod In Minecraft! EPIC MUST SEE M...,One of the coolest mods i have ever seen\n\nMo...,2012-03-09T23:29:03Z,5823927,0,0,PT3M59S,false,hd,9254


In [12]:
#checking for null values
df.isnull().any()

video_id           False
channel_title      False
title              False
description        False
published_at       False
view_count         False
like_Count         False
favourite_Count    False
duration           False
caption            False
defination         False
commentCount       False
dtype: bool

In [13]:
df.dtypes

video_id           object
channel_title      object
title              object
description        object
published_at       object
view_count         object
like_Count         object
favourite_Count    object
duration           object
caption            object
defination         object
commentCount       object
dtype: object

In [14]:
df['like_Count'] = df['like_Count'].apply(pd.to_numeric)
df['favourite_Count'] = df['favourite_Count'].apply(pd.to_numeric)
df['commentCount'] = df['commentCount'].apply(pd.to_numeric)
df['view_count'] = df['view_count'].apply(pd.to_numeric)

In [15]:
# create link
df['video_link'] = df['video_id'].apply(lambda x: f"https://www.youtube.com/watch?v={x}")

In [16]:
#most viewed video
df[df['view_count'] == df['view_count'].max()].video_link

143    https://www.youtube.com/watch?v=se50viFJ0AQ
Name: video_link, dtype: object

In [24]:
df[df['view_count'] == df['view_count'].min()].video_link


459    https://www.youtube.com/watch?v=ttWTSskmxTA
Name: video_link, dtype: object

In [23]:
df['view_count'].describe

<bound method NDFrame.describe of 0       96792587
1      230744065
2       45616652
3       53745586
4      111796791
         ...    
858      1174248
859      1313200
860      1546959
861      5823927
862     29752445
Name: view_count, Length: 863, dtype: int64>

In [25]:
df.columns

Index(['video_id', 'channel_title', 'title', 'description', 'published_at',
       'view_count', 'like_Count', 'favourite_Count', 'duration', 'caption',
       'defination', 'commentCount', 'video_link'],
      dtype='object')

In [52]:
df['Published_date'] = df['published_at'].str.split('T').str[0]

In [53]:
df['Published_time'] = df['published_at'].str.split('T').str[1]

In [55]:
df.drop(['date','time'],axis=1,inplace=True)

In [61]:
len(df[df['Published_time'].str[-1] == 'Z'])

863

In [63]:
df['Published_time'] = df['Published_time'].str.replace('Z','')
df['Published_time']

0      16:30:14
1      17:00:15
2      16:00:01
3      16:00:01
4      16:00:12
         ...   
858    01:59:21
859    23:35:45
860    22:34:11
861    23:29:03
862    22:42:32
Name: Published_time, Length: 863, dtype: object

In [64]:
df['duration']

0         PT40S
1         PT20S
2      PT10M29S
3         PT58S
4         PT18S
         ...   
858       PT31S
859      PT2M6S
860     PT1M30S
861     PT3M59S
862     PT2M37S
Name: duration, Length: 863, dtype: object